In [79]:
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns
import sklearn
import impyute

In [80]:
data = pd.read_csv('credit_card_default_train.csv') 
data.head()


,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
0,A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,...,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
1,A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,...,43530,80811,124590,151818,46200,43530,80811,942,33666,0
2,A4,100K,F,High School,Single,31-45,4,3,2,2,...,0,0,0,0,0,0,0,0,0,1
3,A5,200K,F,Graduate,Single,31-45,2,0,0,0,...,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
4,A6,1M,F,Graduate,Other,31-45,2,2,0,0,...,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [81]:

data["Gender"]=data["Gender"].replace('M',1,regex=True) #Male = 1
data["Gender"]=data["Gender"].replace('F',0,regex=True) #Female =0




data["MARITAL_STATUS"]=data["MARITAL_STATUS"].replace('Other',1,regex=True) #Male = 1
data["MARITAL_STATUS"]=data["MARITAL_STATUS"].replace('Single',0,regex=True) #Female =0


data["EDUCATION_STATUS"]=data["EDUCATION_STATUS"].replace('Other',1,regex=True) #Male = 1
data["EDUCATION_STATUS"]=data["EDUCATION_STATUS"].replace('High School',2,regex=True) #Female =0
data["EDUCATION_STATUS"]=data["EDUCATION_STATUS"].replace('Graduate',3,regex=True) #Female =0


data["AGE"]=data["AGE"].replace('Less than 30',1,regex=True) #Male = 1
data["AGE"]=data["AGE"].replace('31-45',2,regex=True) #Female =0
data["AGE"]=data["AGE"].replace('46-65',3,regex=True) #Male = 1
data["AGE"]=data["AGE"].replace('More than 65',4,regex=True) #Female =0

data

,Client_ID,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
0,A2,1M,1,3,1,2,-1,-1,-1,-1,...,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
1,A3,1M,0,2,1,1,0,-1,-1,-1,...,43530,80811,124590,151818,46200,43530,80811,942,33666,0
2,A4,100K,0,2,0,2,4,3,2,2,...,0,0,0,0,0,0,0,0,0,1
3,A5,200K,0,3,0,2,2,0,0,0,...,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
4,A6,1M,0,3,1,2,2,2,0,0,...,435354,445271,453899,0,20790,16170,17325,16401,17325,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,A29996,1M,0,2,0,2,0,0,0,0,...,203289,72157,36914,19635,46200,11557,7039,11550,2310,0
23996,A29997,400K,0,2,1,2,-1,-1,-1,-1,...,20741,11989,0,4243,8145,20785,298,0,0,0
23997,A29998,100K,0,1,1,2,4,3,2,-1,...,48228,47544,44715,0,0,50820,9702,4620,7161,1
23998,A29999,200K,0,2,0,2,1,-1,0,0,...,121908,27385,113061,198429,7875,2721,4449,122347,4167,1


In [82]:
for  i  in range(data["Balance_Limit_V1"].count()):
    if  type(data["Balance_Limit_V1"][i])==str and data["Balance_Limit_V1"][i][-1] == 'M' :
        data["Balance_Limit_V1"][i] = float(data["Balance_Limit_V1"][i][:-1])*1000
    elif type(data["Balance_Limit_V1"][i])==str and data["Balance_Limit_V1"][i][-1] == 'K' :
        data["Balance_Limit_V1"][i] = float(data["Balance_Limit_V1"][i][:-1])
        
print(data["Balance_Limit_V1"])

C:\Users\Imesh Ekanayake\.conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Imesh Ekanayake\.conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0        1000
1        1000
2         100
3         200
4        1000
         ... 
23995    1000
23996     400
23997     100
23998     200
23999     200
Name: Balance_Limit_V1, Length: 24000, dtype: object


In [87]:

data['Balance_Limit_V1']= data['Balance_Limit_V1'].apply(pd.to_numeric)
Data=data
Data.dtypes

Client_ID              object
Balance_Limit_V1      float64
Gender                  int64
EDUCATION_STATUS        int64
MARITAL_STATUS          int64
AGE                     int64
PAY_JULY                int64
PAY_AUG                 int64
PAY_SEP                 int64
PAY_OCT                 int64
PAY_NOV                 int64
PAY_DEC                 int64
DUE_AMT_JULY            int64
DUE_AMT_AUG             int64
DUE_AMT_SEP             int64
DUE_AMT_OCT             int64
DUE_AMT_NOV             int64
DUE_AMT_DEC             int64
PAID_AMT_JULY           int64
PAID_AMT_AUG            int64
PAID_AMT_SEP            int64
PAID_AMT_OCT            int64
PAID_AMT_NOV            int64
PAID_AMT_DEC            int64
NEXT_MONTH_DEFAULT      int64
dtype: object

In [86]:
Data.describe()#statistical data

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,...,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,...,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,2.400000e+04,24000.000000,2.400000e+04,24000.000000
mean,519.645833,0.602500,1.870042,0.544583,1.862000,-0.014208,-0.130042,-0.158667,-0.215542,-0.261167,...,1.003573e+05,9.377699e+04,9.034068e+04,1.330648e+04,1.386675e+04,1.209312e+04,1.122527e+04,11174.737208,1.230122e+04,0.222083
std,403.520436,0.489391,0.905319,0.498019,0.697717,1.124897,1.200566,1.198875,1.166608,1.131855,...,1.490071e+05,1.417389e+05,1.387237e+05,3.935742e+04,5.648588e+04,4.065924e+04,3.624693e+04,36040.821759,4.233848e+04,0.415656
min,100.000000,0.000000,1.000000,0.000000,1.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-3.927000e+05,-1.878820e+05,-7.844830e+05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000
25%,200.000000,0.000000,1.000000,0.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,5.429000e+03,4.179750e+03,2.913000e+03,2.310000e+03,1.956000e+03,9.010000e+02,6.930000e+02,610.000000,3.070000e+02,0.000000
50%,400.000000,1.000000,2.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.410500e+04,4.186300e+04,3.940900e+04,4.920000e+03,4.646500e+03,4.197000e+03,3.465000e+03,3465.000000,3.465000e+03,0.000000
75%,1000.000000,1.000000,3.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.269750e+05,1.169262e+05,1.144352e+05,1.160500e+04,1.155000e+04,1.062600e+04,9.360000e+03,9411.500000,9.252500e+03,0.000000
max,2500.000000,1.000000,3.000000,1.000000,4.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,2.059564e+06,2.141765e+06,2.221444e+06,2.017905e+06,3.890638e+06,2.069852e+06,1.434510e+06,965557.000000,1.221218e+06,1.000000


In [89]:

highCo=[ 'PAY_JULY', 'PAY_AUG', 'PAY_SEP', 'PAY_OCT',
       'PAY_NOV', 'PAY_DEC', 'PAID_AMT_JULY', 'PAID_AMT_AUG', 'PAID_AMT_SEP',
       'PAID_AMT_OCT', 'PAID_AMT_NOV', 'PAID_AMT_DEC', 'Gender',
       'EDUCATION_STATUS', 'MARITAL_STATUS', 'DUE_AMT_JULY', 'DUE_AMT_AUG',
       'DUE_AMT_SEP', 'DUE_AMT_OCT', 'DUE_AMT_NOV', 'DUE_AMT_DEC', 'AGE','NEXT_MONTH_DEFAULT']

data_set=Data.loc[:,highCo].transpose()

data_set=data_set.values
type(data_set)

numpy.ndarray

In [90]:

data =data_set

train_ratio = 0.6  
limit = int(data.shape[1]*0.6)

print("Data set size; Rows:%d  Samples:%d \nTraining set:%d at %d percent      Test set:%d at %d percent\n\n"%(data.shape[0], data.shape[1], limit, train_ratio*100, data.shape[1]-limit, (1-train_ratio)*100))

x_train = data[0:24, 0:limit] 
y_train = data[-1,   0:limit].reshape(1,-1)

mini_batch = 50
sample_size = x_train.shape[1] 

x_test = data[0:24, limit:]
y_test = data[-1,   limit:].reshape(1,-1)

HL1 = 100
HL2 = 80

HL3 = 400
HL4 = 200
HL5 = 100

HL_last = 1
lr = 0.01


X = tf.placeholder(tf.float64,(x_train.shape[0],None),name="X")
Y = tf.placeholder(tf.float64,(1,None),name="Y")

W1 = tf.Variable(np.random.randn(HL1, x_train.shape[0]),dtype=tf.float64)
W2 = tf.Variable(np.random.randn(HL2, HL1),dtype=tf.float64)
# W3 = tf.Variable(np.random.randn(HL3, HL2),dtype=tf.float64)
# W4 = tf.Variable(np.random.randn(HL4, HL3),dtype=tf.float64)
# W5 = tf.Variable(np.random.randn(HL5, HL4),dtype=tf.float64)
WL = tf.Variable(np.random.randn(1, HL2),dtype=tf.float64)

b1 = tf.Variable(0,dtype=tf.float64)
b2 = tf.Variable(0,dtype=tf.float64)
b3 = tf.Variable(0,dtype=tf.float64)
b4 = tf.Variable(0,dtype=tf.float64)
b5 = tf.Variable(0,dtype=tf.float64)
bL = tf.Variable(0,dtype=tf.float64)



Z1 = tf.add(tf.matmul(W1,X),b1)
A1 = tf.nn.relu(Z1)

Z2 = tf.add(tf.matmul(W2,A1),b2)
A2 = tf.nn.relu(Z2)

# Z3 = tf.add(tf.matmul(W3,A2),b3)
# A3 = tf.nn.relu(Z3)

# Z4 = tf.add(tf.matmul(W4,A3),b4)
# A4 = tf.nn.relu(Z4)

# Z5 = tf.add(tf.matmul(W5,A4),b5)
# A5 = tf.nn.relu(Z5)

Z_out = tf.add(tf.matmul(WL,A2),bL)
A_out = tf.nn.sigmoid(Z_out)


cost = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.transpose(Y) , logits= tf.transpose(A_out))
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
ses = tf.Session()

ses.run(tf.global_variables_initializer())

print(x_train[10])
for i in range(10000):
  for j  in range(0,sample_size, mini_batch):
    batch_x = x_train[:, j:j + mini_batch]
    batch_y = y_train[0, j:j + mini_batch].reshape(1,mini_batch)
        
    ses.run([cost,train], feed_dict={X: batch_x, Y:batch_y})
    
    out = ses.run(A_out,feed_dict={X: x_test})
    cos = np.sum(ses.run(cost, feed_dict={X: x_test, Y:y_test}))/y_test.shape[1]
    out = np.around(out)
    
    correct_preds = np.sum(out == y_test)
    accuracy = (correct_preds/y_test.shape[1])
    print("Accuracy at epoch %d : %f    cost : %f"%(i,accuracy,cos))




Data set size; Rows:23  Samples:24000 
Training set:14400 at 60 percent      Test set:9600 at 40 percent


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[    0   942     0 ...    51  3262 11550]
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.754479    cost : 0.719853
Accuracy at epoch 0 : 0.755417    cost : 0.719215
Accuracy at epoch 0 : 0.755313    cost : 0.719228
Accuracy at epoch 0 : 0.756042    cost : 0.718760
Accuracy at epoch 0 : 0.756250    cost : 0.718612
Accuracy at epoch 0 : 0.756875    cost : 0.718280
Accurac

Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580
Accuracy at epoch 0 : 0.759271    cost : 0.716580


Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580


Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580
Accuracy at epoch 1 : 0.759271    cost : 0.716580


Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580
Accuracy at epoch 2 : 0.759271    cost : 0.716580


...

Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at 

Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at epoch 205 : 0.793125    cost : 0.693535
Accuracy at 

Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at 

Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 206 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at epoch 207 : 0.793125    cost : 0.693535
Accuracy at 

KeyboardInterrupt: 